In [67]:
import os
import wave
import contextlib
from pathlib import Path
from google.colab import files
from os import listdir
import sys

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras import backend
from numpy import zeros
from sklearn.metrics import fbeta_score
from numpy import ones
from numpy import asarray
from numpy import savez_compressed
import pandas as pd
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.io import wavfile as wav
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
import numpy as np
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Reshape, Dense, LSTM, add, concatenate, Dropout, Lambda, Flatten, GRU, LeakyReLU
from keras.models import Model
import keras.backend as K
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from pandas import read_csv

In [68]:
sampling_rate=16000
audio_duration=2
n_classes=3022
n_folds_=10
learning_rate=0.0001
max_epochs=50
n_mfcc=20
audio_length = sampling_rate * audio_duration
dim = (audio_length, 1)
batch_size=64

In [69]:
#best case, the dataset is in drive/MyDrive/datasets
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
def audio_norm(data):
    max_data = np.max(data)
    min_data = np.min(data)
    data = (data-min_data)/(max_data-min_data+1e-6)
    return data-0.5

In [71]:
#filter files that actually have sound by their size (not the best)
actualFiles = []
counter = 0
directory = r'drive/MyDrive/DISTANCE/'
for filename in os.listdir(directory):
  if(filename.endswith(".wav")):
    size = Path(directory + filename).stat().st_size
    if(size >= 59):
      actualFiles.append(directory + filename)
  counter += 1

In [72]:
train = pd.read_csv("drive/MyDrive/metadata.csv")
X = pd.read_csv("drive/MyDrive/metadata.csv", usecols = ["fileName", "distance"])
y = pd.read_csv("drive/MyDrive/metadata.csv", usecols = ["label"])

#test = pd.read_csv("drive/MyDrive/datasets/FSDKaggle2018.meta/test_post_competition_scoring_clips.csv")
train.head()

,fileName,distance,label
0,10000.wav,2,"Breathing,Respiratory_sounds"
1,100005.wav,2,"Rattle_(instrument),Percussion,Musical_instrum..."
2,10000.wav,2,"Breathing,Respiratory_sounds"
3,100005.wav,2,"Rattle_(instrument),Percussion,Musical_instrum..."
4,100007.wav,2,"Rattle_(instrument),Percussion,Musical_instrum..."


In [73]:
#ONLY RUN THIS ONCE, or do the second cell again
LABELS = list(train.label.unique())
#LABELS
label_idx = {label: i for i, label in enumerate(LABELS)}
#label_idx
#interestingly enough, Hi-hat is 0 for some reason
train.set_index("fileName", inplace=True)
#set the index row for train to the name of the file (you can create a new row instead)
#train.head()
#same for test
train["label_idx"] = train.label.apply(lambda x: label_idx[x])
#create row where every element has the amount of times that label is used

In [74]:
def getModel():
    input_length = audio_length

    inp = Input(shape=(dim[0], dim[1],1))
    x = Conv2D(32, (3,3), padding="same")(inp)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(32, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(32, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(32, (3,3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D()(x)

    x = Flatten()(x)
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    out = Dense(n_classes, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(learning_rate)

    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['acc'])
    return model

In [75]:
dim = (n_mfcc, 1+(int(np.floor(audio_length/512))), 1)

In [76]:
def prepare_data(df, data_dir):
    X = np.empty(shape=(df.shape[0], dim[0], dim[1], 1))
    input_length = audio_length
    #create librosa file
    for i, fname in enumerate(data_dir):
        file_path = fname
        data, _ = librosa.core.load(file_path, sr=sampling_rate, res_type="kaiser_fast")


        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, input_length - len(data) - offset), "constant")
        #extract mfcc features
        data = librosa.feature.mfcc(data, sr=sampling_rate, n_mfcc=n_mfcc)
        data = np.expand_dims(data, axis=-1)
        #save them and do it for each file, return X
        X[i,] = data
    return X

In [77]:
print(train.head())
print(actualFiles[0])
#this takes a lot of time
X_train = prepare_data(train, actualFiles)
#'drive/MyDrive/datasets/FSDKaggle2018.audio_train/'

            distance  ... label_idx
fileName              ...          
10000.wav          2  ...         0
100005.wav         2  ...         1
10000.wav          2  ...         0
100005.wav         2  ...         1
100007.wav         2  ...         1

[5 rows x 3 columns]
drive/MyDrive/DISTANCE/119312.wav


In [78]:
y_train = to_categorical(train.label_idx, num_classes=3022)

In [79]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

#some kind of normalization
X_train = (X_train - mean)/std

In [80]:
print(X_train.shape)
print(y_train.shape)

(3022, 20, 63, 1)
(3022, 3022)


In [81]:
PREDICTION_FOLDER = "predictions_1d_conv"
if not os.path.exists(PREDICTION_FOLDER):
    os.mkdir(PREDICTION_FOLDER)

In [82]:
from sklearn.model_selection import KFold
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                            ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
#kfold this time for simplicity
#the idea is the same as the previous one
kf = KFold(n_splits=2)
for i, (train_index, test_index) in enumerate(kf.split(X_train)):
  X_t, X_te = X_train[train_index], X_train[test_index]
  y_t, y_te = y_train[train_index], y_train[test_index]
  print("#"*50)
  print("Fold: ", i)
  model = getModel()
  checkpoint = ModelCheckpoint('best_%d.h5'%i, monitor='val_loss', verbose=1, save_best_only=True)
  early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
  tb = TensorBoard(log_dir='./logs/' + PREDICTION_FOLDER + '/fold_%i'%i, write_graph=True)
  callbacks_list = [checkpoint, early, tb]
  history = model.fit(X_t, y_t, validation_data=(X_te, y_te), callbacks=callbacks_list, 
                        batch_size=64, epochs=30)
  model.load_weights('best_%d.h5'%i)
  predictions = model.predict(X_train, batch_size=64, verbose=1)
  np.save(PREDICTION_FOLDER + "/train_predictions_%d.npy"%i, predictions)

##################################################
Fold:  0
Epoch 1/30
24/24 [==============================] - 13s 461ms/step - loss: 7.9959 - acc: 1.6954e-04 - val_loss: 8.0074 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 8.00735, saving model to best_0.h5
Epoch 2/30
24/24 [==============================] - 10s 429ms/step - loss: 7.8510 - acc: 0.0324 - val_loss: 7.9845 - val_acc: 0.0079

Epoch 00002: val_loss improved from 8.00735 to 7.98454, saving model to best_0.h5
Epoch 3/30
24/24 [==============================] - 10s 432ms/step - loss: 7.7177 - acc: 0.1640 - val_loss: 7.9518 - val_acc: 0.0126

Epoch 00003: val_loss improved from 7.98454 to 7.95180, saving model to best_0.h5
Epoch 4/30
24/24 [==============================] - 10s 430ms/step - loss: 7.5837 - acc: 0.1825 - val_loss: 7.9163 - val_acc: 0.0126

Epoch 00004: val_loss improved from 7.95180 to 7.91635, saving model to best_0.h5
Epoch 5/30
24/24 [==============================] - 10s 430ms/step - los